In [ ]:
"""
ResNet-50 Implementation in Keras

This code implements the ResNet-50 architecture, a deep convolutional neural network designed to address the challenges of training very deep networks. The key innovation of ResNet-50 is the use of residual connections, which help mitigate the vanishing gradient problem and allow for more efficient training of deeper models.

Key Features:
- **Residual Blocks**: Each block contains skip connections (shortcut connections) that allow the network to learn residual functions rather than direct mappings.
- **Identity Blocks**: These blocks maintain the same input and output dimensions, enabling the network to deepen without significant computational overhead.
- **Global Average Pooling**: The model ends with a global average pooling layer that reduces spatial dimensions to a 1D vector, followed by a sigmoid activation for binary classification.
- **Keras Implementation**: The model is implemented using Keras with TensorFlow backend, making it easy to train, modify, and experiment with.

Use this model for:
- **Image Classification**: Apply ResNet-50 for tasks like object recognition or image classification on large datasets such as ImageNet.
- **Transfer Learning**: Use the pre-trained ResNet-50 model for fine-tuning on custom datasets.

This implementation provides the flexibility to build and experiment with the ResNet-50 architecture for both educational and research purposes.

"""


In [37]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Conv2D, Activation, concatenate, Conv2DTranspose,MaxPooling2D,Input,Cropping2D,Lambda,Dropout,BatchNormalization,GlobalAvgPool2D,AveragePooling2D,Add,ZeroPadding2D
from tensorflow.keras.models import Model

In [40]:
# Identity Block
def identity_block(input, filter_no1, filter_no2, filter_no3):
    # First convolution (1x1)
    s1 = Conv2D(filter_no1, kernel_size=(1, 1), padding='same')(input)
    s1 = BatchNormalization(axis=3)(s1)
    s1 = Activation('relu')(s1)

    # Second convolution (3x3)
    s2 = Conv2D(filter_no2, kernel_size=(3, 3), padding='same')(s1)
    s2 = BatchNormalization(axis=3)(s2)
    s2 = Activation('relu')(s2)

    # Third convolution (1x1)
    s3 = Conv2D(filter_no3, kernel_size=(1, 1), padding='same')(s2)
    s3 = BatchNormalization(axis=3)(s3)

    # Add shortcut
    s3 = Add()([s3, input])
    s3 = Activation('relu')(s3)

    return s3

# Residual Block
def residual_block(input, filter_no1, filter_no2, filter_no3):
    # First convolution (1x1), with strides for downsampling if needed
    stride = (2, 2)
    s1 = Conv2D(filter_no1, kernel_size=(1, 1), strides=stride, padding='same')(input)
    s1 = BatchNormalization(axis=3)(s1)
    s1 = Activation('relu')(s1)

    # Second convolution (3x3)
    s2 = Conv2D(filter_no2, kernel_size=(3, 3), padding='same')(s1)
    s2 = BatchNormalization(axis=3)(s2)
    s2 = Activation('relu')(s2)

    # Third convolution (1x1)
    s3 = Conv2D(filter_no3, kernel_size=(1, 1), padding='same')(s2)
    s3 = BatchNormalization(axis=3)(s3)

    # Shortcut connection
    shortcut = Conv2D(filter_no3, kernel_size=(1, 1), strides=stride, padding='same')(input)
    shortcut = BatchNormalization(axis=3)(shortcut)

    # Add shortcut
    s3 = Add()([s3, shortcut])
    s3 = Activation('relu')(s3)

    return s3

In [42]:
# Model Building
input = Input((256, 256, 3))
scaled_input = Lambda(lambda x: x / 255)(input)

# Initial layers
x = ZeroPadding2D((3, 3))(scaled_input)
x = Conv2D(64, (7, 7), strides=(2, 2), padding='valid')(x)
x = BatchNormalization(axis=3)(x)
x = Activation('relu')(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

# Stage 1
x = residual_block(x, 64, 64, 256)
x = identity_block(x, 64, 64, 256)
x = identity_block(x, 64, 64, 256)

# Stage 2
x = residual_block(x, 128, 128, 512)
x = identity_block(x, 128, 128, 512)
x = identity_block(x, 128, 128, 512)
x = identity_block(x, 128, 128, 512)

# Stage 3
x = residual_block(x, 256, 256, 1024)
x = identity_block(x, 256, 256, 1024)
x = identity_block(x, 256, 256, 1024)
x = identity_block(x, 256, 256, 1024)
x = identity_block(x, 256, 256, 1024)
x = identity_block(x, 256, 256, 1024)

# Stage 4
x = residual_block(x, 512, 512, 2048)
x = identity_block(x, 512, 512, 2048)
x = identity_block(x, 512, 512, 2048)

# Average Pooling and Output layer
x = GlobalAvgPool2D()(x)
output = Dense(1, activation='sigmoid')(x)

# Compile Model
model = Model(inputs=input, outputs=output)
model.compile(optimizer='adam', metrics=['accuracy'], loss='binary_crossentropy')

model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14            │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_12 (Lambda)        │ (None, 256, 256, 3)    │              0 │ input_layer_14[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_4          │ (None, 262, 262, 3)    │              0 │ lambda_12[0][0]        │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_251 (Conv2D)       │ (None, 128, 128, 64)   │          9,472 │ zero_padding2d_4[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_242   │ (None, 128, 128, 64)   │            256 │ conv2d_251[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_221            │ (None, 128, 128, 64)   │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_12          │ (None, 64, 64, 64)     │              0 │ activation_221[0][0]   │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_252 (Conv2D)       │ (None, 32, 32, 64)     │          4,160 │ max_pooling2d_12[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_243   │ (None, 32, 32, 64)     │            256 │ conv2d_252[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_222            │ (None, 32, 32, 64)     │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_253 (Conv2D)       │ (None, 32, 32, 64)     │         36,928 │ activation_222[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_244   │ (None, 32, 32, 64)     │            256 │ conv2d_253[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_223            │ (None, 32, 32, 64)     │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_254 (Conv2D)       │ (None, 32, 32, 256)    │         16,640 │ activation_223[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_255 (Conv2D)       │ (None, 32, 32, 256)    │         16,640 │ max_pooling2d_12[0][0] │
├──────────────────────

 Total params: 23,589,761 (89.99 MB)

 Trainable params: 23,536,641 (89.79 MB)

 Non-trainable params: 53,120 (207.50 KB)